In [3]:
import sys
import os
import numpy as np
import pandas as pd
import gzip
from script import days_in_month, hours_in_day

# twitter_geography Folder Analysis

## twitter_geography/2015

In [2]:
geo_path = "/srv/data/twitter_geography/2015/"
files = [f for f in os.listdir(geo_path) if os.path.isfile(os.path.join(geo_path, f))]
print("There are a total of", len(files), "files in twitter_geography/2015.")

There are a total of 8759 files in twitter_geography/2015.


### Sample Dataframe 

In [3]:
with gzip.open(''.join([geo_path, "geography_2015_1_10_00.csv.gz"])) as f:
    geo_posts = pd.read_csv(f, sep="\t")
        
geo_posts.head()

,message_id,OBJECTID,ID_0,NAME_0,ISO,ID_1,NAME_1,ID_2,NAME_2
0,553716693625614336,41431,244,United States,USA,5,California,202,Los Angeles
1,553703608068632576,41050,242,United Kingdom,GBR,1,England,13,Cambridgeshire
2,553708287170859008,43799,244,United States,USA,44,Texas,2570,Collin
3,553717365678944256,43857,244,United States,USA,44,Texas,2628,Harris
4,553708727346286592,18411,106,Indonesia,IDN,32,Sumatera Selatan,413,Prabumulih


### Summary Dataframe 

In [19]:
def num_posts_on_day(path, prefix, year, month, day):
    num_posts = 0
    day_path = ''.join([path, prefix, "_", str(year), "_", str(month), "_", str(day).zfill(2)])
    for hour in hours_in_day():
        try:
            with gzip.open(''.join([day_path, "_", hour, ".csv.gz"])) as f:
                posts = pd.read_csv(f, sep="\t")
                num_posts += len(posts)
        except FileNotFoundError:
            print(''.join([day_path, "_", hour, ".csv.gz"]), "does not exist.")
    return num_posts

In [21]:
df = []
for month in range(1, 13):
    for day in range(1, days_in_month(month, 2015) + 1):
        df.append([2015, month, day, num_posts_on_day(geo_path, "geography", 2015, month, day)])

KeyboardInterrupt: 

In [ ]:
summary_df = pd.DataFrame(data=df,    # values 
              columns=["year", "month", "day", "num_posts"])
summary_df

In [ ]:
summary_df.to_csv("twitter_geography_2015_summary.csv")

# twitter_sentiment Folder Analysis

## twitter_sentiment/2015 

In [4]:
sent_path = "/srv/data/twitter_sentiment/2015/"
files = [f for f in os.listdir(sent_path) if os.path.isfile(os.path.join(sent_path, f))]
print("There are a total of", len(files), "files in twitter_sentiment/2015.")

There are a total of 8760 files in twitter_sentiment/2015.


### Sample Dataframe 

In [5]:
with gzip.open(''.join([sent_path, "bert_sentiment_2015_1_10_00.csv.gz"])) as f:
    sent_posts = pd.read_csv(f, sep="\t")
        
sent_posts.head()

,message_id,score
0,553702825499578369,0.405586
1,553703850851729409,0.565157
2,553704279773814784,0.401424
3,553704506689863680,0.536007
4,553705203728666624,0.460751


### Summary Dataframe

In [ ]:
df = []
for month in range(1, 13):
    for day in range(1, days_in_month(month, 2015) + 1):
        df.append([2015, month, day, num_posts_on_day(path, "bert_sentiment", 2015, month, day)])

In [ ]:
summary_df = pd.DataFrame(data=df,    # values 
             columns=["year", "month", "day", "num_posts"]) 

In [ ]:
summary_df

In [ ]:
summary_df.to_csv("twitter_sentiment_2015_summary.csv")

# Master Dataframe 

In [4]:
def num_posts_on_day_two_df(path_1, prefix_1, path_2, prefix_2, year, month, day):
    num_posts_1 = 0
    num_posts_2 = 0
    num_posts_common = 0
    day_path_1 = ''.join([path_1, prefix_1, "_", str(year), "_", str(month), "_", str(day).zfill(2)])
    day_path_2 = ''.join([path_2, prefix_2, "_", str(year), "_", str(month), "_", str(day).zfill(2)])
    common_posts_by_hour = []
    for hour in hours_in_day():
        try:
            with gzip.open(''.join([day_path_1, "_", hour, ".csv.gz"])) as f:
                posts_1 = pd.read_csv(f, sep="\t")
                num_posts_1 += len(posts_1)
        except FileNotFoundError:
            print(''.join([day_path_1, "_", hour, ".csv.gz"]), "does not exist.")
            continue
        try:
            with gzip.open(''.join([day_path_2, "_", hour, ".csv.gz"])) as f:
                posts_2 = pd.read_csv(f, sep="\t")
                num_posts_2 += len(posts_2)
        except FileNotFoundError:
            print(''.join([day_path_2, "_", hour, ".csv.gz"]), "does not exist.")
            continue
        common_posts_by_hour.append(pd.merge(posts_1, posts_2, on="message_id", how="inner"))
        num_posts_common = len(pd.concat(common_posts_by_hour, ignore_index=True))
    return num_posts_1, num_posts_2, num_posts_common

### Sample Call

In [24]:
num_posts_on_day_two_df(geo_path, "geography", sent_path, "bert_sentiment", 2015, 12, 10)

(977941, 1021682, 977941)

## 2015 

In [30]:
df = []
for month in range(1, 13):
    for day in range(1, days_in_month(month, 2015) + 1):
        df.append([2015, month, day] + list(num_posts_on_day_two_df(geo_path, "geography", sent_path, "bert_sentiment", 2015, month, day)))

/srv/data/twitter_geography/2015/geography_2015_6_02_00.csv.gz does not exist.


In [31]:
master_summary_df = pd.DataFrame(data=df,    # values 
             columns=["year", "month", "day", "num_geo_posts", "num_sent_posts", "num_common_posts"]) 
master_summary_df

,year,month,day,num_geo_posts,num_sent_posts,num_common_posts
0,2015,1,1,6772377,7033347,6772377
1,2015,1,2,6798624,7040550,6798623
2,2015,1,3,6939489,7188683,6939489
3,2015,1,4,7349366,7609473,7349365
4,2015,1,5,7170502,7414078,7170501
...,...,...,...,...,...,...
360,2015,12,27,1119233,1167184,1119233
361,2015,12,28,1030613,1078400,1030613
362,2015,12,29,1008498,1053860,1008498
363,2015,12,30,1010463,1055810,1010463


In [32]:
master_summary_df.to_csv("master_2015_summary.csv")

## 2018

In [18]:
# bug fixing
for hour in range(0, 24):
    with gzip.open(''.join([geo_path_2018, "geography","_", str(2018), "_", str(10), "_", str(13), "_", str(hour).zfill(2), ".csv.gz"])) as f:
        dumb_df = pd.read_csv(f, sep="\t")


EmptyDataError: No columns to parse from file

In [12]:
geo_path_2018 = "/srv/data/twitter_geography/2018/"
sent_path_2018 = "/srv/data/twitter_sentiment/2018/"

In [13]:
num_posts_on_day_two_df(geo_path_2018, "geography", sent_path_2018, "bert_sentiment", 2018, 1, 10)

(380049, 399214, 380049)

In [15]:
df = []
for month in range(1, 13):
    for day in range(1, days_in_month(month, 2018) + 1):
        print(month, day)
        df.append([2018, month, day] + list(num_posts_on_day_two_df(geo_path_2018, "geography", sent_path_2018, "bert_sentiment", 2018, month, day)))

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
4 28
4 29
4 30
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
5 25
5 26
5 27
5 28
5 29
5 30
5 31
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
6 22
6 23
6 24
6 25
6 26
6 27
6 28
6 29
6 30
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19
7 20
7 21
7 22
7 23
7 24
7 25
7 26
7 27
7 28
7 29
7 30
7 31
8 1

EmptyDataError: No columns to parse from file

In [ ]:
master_summary_df_2018 = pd.DataFrame(data=df,    # values 
             columns=["year", "month", "day", "num_geo_posts", "num_sent_posts", "num_common_posts"]) 
master_summary_df_2018

In [ ]:
master_summary_df_2018.to_csv("master_2018_summary.csv")

## 1/10, 2012-2021 

In [6]:
df = []
for year in range(2012, 2022):
    geo_path = ''.join(["/srv/data/twitter_geography/", str(year), "/"])
    sent_path = ''.join(["/srv/data/twitter_sentiment/", str(year), "/"])
    df.append([year, 1, 10] + list(num_posts_on_day_two_df(geo_path, "geography", sent_path, "bert_sentiment", year, 1, 10)))

/srv/data/twitter_geography/2013/geography_2013_1_10_22.csv.gz does not exist.
/srv/data/twitter_geography/2021/geography_2021_1_10_23.csv.gz does not exist.


In [7]:
master_summary_df_1_10_2012_2021 = pd.DataFrame(
    data=df,
    columns=["year", "month", "day", "num_geo_posts", "num_sent_posts", "num_common_posts"]
)
master_summary_df_1_10_2012_2021

,year,month,day,num_geo_posts,num_sent_posts,num_common_posts
0,2012,1,10,20336,21250,20305
1,2013,1,10,4722382,5007452,4705284
2,2014,1,10,3710765,3945108,3675219
3,2015,1,10,6717684,6972698,6717684
4,2016,1,10,633117,667918,633117
5,2017,1,10,545354,568887,545354
6,2018,1,10,380049,399214,380049
7,2019,1,10,3589493,3892095,3589490
8,2020,1,10,3571909,3775946,3489357
9,2021,1,10,2747852,2989402,2747849


In [11]:
master_summary_df_1_10_2012_2021.to_csv('../output/master_2012_2021_1_10_summary.csv')

## Method to Run 2012 - 2020 

In [5]:
def generate_master_df_year(year):
    geo_path = ''.join(["/srv/data/twitter_geography/", str(year), "/"])
    sent_path = ''.join(["/srv/data/twitter_sentiment/", str(year), "/"])
    df = []
    for month in range(1, 13):
        for day in range(1, days_in_month(month, year) + 1):
            df.append([year, month, day] + list(num_posts_on_day_two_df(geo_path, "geography", sent_path, "bert_sentiment", year, month, day)))
    master_summary_df = pd.DataFrame(data=df,    # values 
             columns=["year", "month", "day", "num_geo_posts", "num_sent_posts", "num_common_posts"]) 
    master_summary_df.to_csv(''.join(["../output/master_", str(year), "_summary.csv"]))

In [ ]:
for year in range(2012, 2021):
    generate_master_df_year(year)

/srv/data/twitter_geography/2012/geography_2012_4_01_07.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_01_08.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_01_09.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_01_11.csv.gz does not exist.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



/srv/data/twitter_geography/2012/geography_2012_4_01_18.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_01_19.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_01_20.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_01_21.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_01_22.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_01_23.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_02_01.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_02_02.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_02_04.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_02_06.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_02_07.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_02_08.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_02

/srv/data/twitter_geography/2012/geography_2012_4_08_06.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_08.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_09.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_10.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_11.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_12.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_13.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_14.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_16.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_17.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_19.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_08_20.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_09

/srv/data/twitter_geography/2012/geography_2012_4_16_06.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_07.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_08.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_09.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_10.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_11.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_14.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_15.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_18.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_19.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_16_23.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_17_00.csv.gz does not exist.
/srv/data/twitter_geography/2012/geography_2012_4_17

...

/srv/data/twitter_geography/2013/geography_2013_7_15_03.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_15_09.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_15_11.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_15_12.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_15_13.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_15_16.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_15_19.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_15_23.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_16_00.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_16_05.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_16_07.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_16_10.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_7_16

/srv/data/twitter_geography/2013/geography_2013_8_01_19.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_02_02.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_02_07.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_02_11.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03_00.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03_02.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03_05.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03_06.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03_07.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03_10.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03_12.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03_13.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_03

/srv/data/twitter_geography/2013/geography_2013_8_17_17.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_17_18.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_18_05.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_18_07.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_18_11.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_18_13.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_18_23.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_19_06.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_19_07.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_19_09.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_19_15.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_20_01.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_8_20

/srv/data/twitter_geography/2013/geography_2013_9_02_21.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_03_05.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_03_08.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_03_12.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_03_14.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_03_20.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_04_03.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_04_04.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_04_06.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_04_07.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_04_15.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_04_18.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_05

/srv/data/twitter_geography/2013/geography_2013_9_20_12.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_20_14.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_20_17.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_20_21.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_20_22.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_21_11.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_21_17.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_21_22.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_21_23.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_22_08.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_22_11.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_22_21.csv.gz does not exist.
/srv/data/twitter_geography/2013/geography_2013_9_23

In [ ]:
generate_master_df_year(2014)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



/srv/data/twitter_geography/2014/geography_2014_1_19_14.csv.gz does not exist.
/srv/data/twitter_geography/2014/geography_2014_1_19_23.csv.gz does not exist.
/srv/data/twitter_geography/2014/geography_2014_1_20_05.csv.gz does not exist.
/srv/data/twitter_geography/2014/geography_2014_1_20_08.csv.gz does not exist.
/srv/data/twitter_geography/2014/geography_2014_1_20_09.csv.gz does not exist.
/srv/data/twitter_geography/2014/geography_2014_1_20_13.csv.gz does not exist.
/srv/data/twitter_geography/2014/geography_2014_1_21_14.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_17_22.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_17_23.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_18_00.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_18_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_18_02.csv.gz does not exist.
/srv/data/twitter_sentiment

/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_21_17.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_21_18.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_21_19.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_21_20.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_21_21.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_21_22.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_21_23.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_22_00.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_22_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_22_02.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_22_03.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_22_04.csv.gz does not

/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_25_19.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_25_20.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_25_21.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_25_22.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_25_23.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_26_00.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_26_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_26_02.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_26_03.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_26_04.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_26_05.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_26_06.csv.gz does not

/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_29_21.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_29_22.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_29_23.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_00.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_02.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_03.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_04.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_05.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_06.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_07.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_4_30_08.csv.gz does not

...

/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_11_17.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_11_18.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_11_19.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_11_20.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_11_21.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_11_22.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_11_23.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_12_00.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_12_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_12_02.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_12_03.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_12_04.csv.gz does not

/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_15_19.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_15_20.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_15_21.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_15_22.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_15_23.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_16_00.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_16_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_16_02.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_16_03.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_16_04.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_16_05.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_16_06.csv.gz does not

/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_19_21.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_19_22.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_19_23.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_00.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_02.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_03.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_04.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_05.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_06.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_07.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_20_08.csv.gz does not

/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_23_23.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_00.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_02.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_03.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_04.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_05.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_06.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_07.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_08.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_09.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_24_10.csv.gz does not

/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_01.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_02.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_03.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_04.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_05.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_06.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_07.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_08.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_09.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_10.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_11.csv.gz does not exist.
/srv/data/twitter_sentiment/2014/bert_sentiment_2014_9_28_12.csv.gz does not

In [ ]:
generate_master_df_year(2016)